In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import math
from scipy.signal import savgol_filter
import json
from datetime import datetime
import statsmodels.api as sm
from scipy.stats import pearsonr
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import StandardScaler
from helper_functions import *

In [ ]:
interventions = load_interventions()
d = load_aggregated_timeseries()

In [ ]:
data, df_code = choose_restrictiveness("All", "Yes")
plot_percent_pageviews(d, df_code, interventions)

In [ ]:
plot_normalized_percent_pageviews(d, df_code, interventions, 2.5)

In [ ]:
data, df_code = choose_restrictiveness("Restrictive", "Yes")
plot_normalized_percent_pageviews(d, df_code, interventions, 2)

In [ ]:
data, df_code = choose_restrictiveness("Semi-Restrictive", "Yes")
plot_normalized_percent_pageviews(d, df_code, interventions, 2.5)

In [ ]:
data, df_code = choose_restrictiveness("Unrestrictive", "Yes")
plot_normalized_percent_pageviews(d, df_code, interventions, 2)

In [ ]:
data, df_code = choose_restrictiveness("All", "Yes")
plot_mobility(d, df_code, interventions)